This program is used to compute the impact that the chip tilt has on thermal resistance. The chip is defined as an infinitely thin rectangle centered at the origin. First, the chip is defined in 3-D space, and given a tilt according to the values of phi and theta. Phi and theta are defined to be the polar angles of a unit normal vector on the surface of the chip, at the origin. Then, the locations of the chip corners are derived. Then, a set of boundary conditions are found based on these corners. Finally, the thermal resistance is found by taking an integral over the solder volume underneath the chip.

Start by defining a function for converting between spherical and cartesian.

In [1]:
import sympy as sp
from IPython.display import display, Math
import plotly.graph_objects as go
import numpy as np

In [2]:
def spherical_to_cartesian(rho, phi, theta):
    x = rho * sp.sin(phi) * sp.cos(theta)
    y = rho * sp.sin(phi) * sp.sin(theta)
    z = rho * sp.cos(phi)
    return [x, y, z]

Make a function for fast display

In [3]:
def disp(expression):
    display(Math(sp.latex(expression)))

Now, define the symbols used.

In [4]:
# define the chip tilt in terms of a normal vector to the surface
phi, theta = sp.symbols('φ θ', positive=True)

# define the width and height of the chip
w, h = sp.symbols('w h', positive=True)

# define the four vectors to the chip corners
A, B, C, D = sp.symbols(r'\mathbf{A} \mathbf{B} \mathbf{C} \mathbf{D}')

# define the length of the vectors in terms of rho
rho = sp.symbols('ρ', positive=True)
rho_ex = sp.sqrt((h/2)**2 + (w/2)**2)

# make the normal vector
n = sp.Matrix(spherical_to_cartesian(1, phi, theta))

Define the first vector, A, as having the same value of theta as the normal vector.

In [5]:
phi_a = phi + sp.pi/2
theta_a = theta
A = sp.Matrix(spherical_to_cartesian(rho, phi_a, theta_a))

The vector C is just the opposite of A.

In [6]:
C = -1*A

The B vector has 3 constrains. First, it has a length of rho. Second, it is at a right angle to the normal vector n. Third, it forms a specific angle with the A vector, determined by the chip dimensions w and h. Set up a system of equations from these constraints.

In [7]:
bx, by, bz = sp.symbols('b_x b_y b_z')
B = sp.Matrix([bx, by, bz])

A_B_angle = 2 * sp.atan(w/h)

# constraint 1
constraint_1 = sp.Eq(bx**2 + by**2 + bz**2, rho**2)

# constraint 2
constraint_2 = sp.Eq(B.dot(n), 0)

# constraint 3
constraint_3 = sp.Eq(B.dot(A) / (rho**2), sp.cos(A_B_angle))

# solve them
solutions = sp.solve((constraint_1, constraint_2, constraint_3), (bx, by, bz))

B_1 = sp.Matrix(solutions[0])
B_2 = sp.Matrix(solutions[1])

There are 2 solutions. They are both valid, they just define the chip to be at different rotations along the z-axis. The first solution is chosen here. Now, D can be defined as just the opposite of B.

In [38]:
B = B_1
D = -1*B

Make a function to plot the vectors for visualizaiton.

In [58]:
def plot_vectors(list_of_vectors, subs):
    """
    Subs is a dict containing all of the desired  substitutions, for numerical computation.
    """
    fig = go.Figure()
    origin = np.array([0, 0, 0])
        
    for vector in list_of_vectors:
        length = len(vector)
        if length < 3:
            vector = sp.Matrix(list(vector) + [0] * (3 - length))
            
        evaluated_vec = vector.subs(subs)
        #disp(evaluated_vec.evalf())
        vector_np = np.array(evaluated_vec).astype(np.float64)
        U = vector_np[0, 0]
        V = vector_np[1, 0]
        W = vector_np[2, 0]
        fig.add_trace(go.Cone(x=[origin[0]], y=[origin[1]], z=[origin[2]], u=[U], v=[V], w=[W], sizemode='raw', sizeref=2, showscale=True, anchor='tail', colorscale='Portland', cmin=1, cmax=5))

    fig.update_layout(title='3D Vector Plot',
                  scene=dict(xaxis_title='X',
                             yaxis_title='Y',
                             zaxis_title='Z'))

    fig.show()       
        

In [80]:
h_val = 5
w_val = 5
subs = {theta:0, phi:1.0, h:h_val, w:w_val, rho:sp.sqrt((h_val/2)**2 + (w_val/2)**2)}
plot_vectors([A, B, C, D], subs)

Now, the bounds of integration will be found by projecting each of the corners onto the x-y plane. From here, a rotation will be applied in order to align the chip for proper integration.

In [85]:
A2 = sp.Matrix([A[0], A[1]])
B2 = sp.Matrix([B[0], B[1]])
C2 = sp.Matrix([C[0], C[1]])
D2 = sp.Matrix([D[0], D[1]])

#plot_vectors([A2, B2, C2, D2], subs)

# align the A-B segment to be parallel to the x-axis
x_hat = sp.Matrix([1, 0])
AB_segment = A - B
rotation_angle = sp.atan(AB_segment[1] / AB_segment[0])

Create rotation matrices for the process of rotating the 2D and 3D objects about the Z-axis.

In [86]:
R2_rotator = sp.Matrix([[sp.cos(rotation_angle), -sp.sin(rotation_angle)], [sp.sin(rotation_angle), sp.cos(rotation_angle)]])

R3_rotator = sp.Matrix([[sp.cos(rotation_angle), -sp.sin(rotation_angle), 0], [sp.sin(rotation_angle), sp.cos(rotation_angle), 0], [0, 0, 1]])

Ar = R3_rotator * A
Br = R3_rotator * B
Cr = R3_rotator * C
Dr = R3_rotator * D

A2r = R2_rotator * A2
B2r = R2_rotator * B2
C2r = R2_rotator * C2
D2r = R2_rotator * D2

plot_vectors([A2, B2], subs)
plot_vectors([A2r, B2r], subs)
plot_vectors([Ar, Br], subs)